<a href="https://colab.research.google.com/github/jrw99/Deep-Learning/blob/main/Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [20]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# mount drive so colab can read/write files
from google.colab import drive
drive.mount('/content/drive')

application_df = pd.read_csv("/content/drive/MyDrive/Homework/Deep-Learning/Resources/charity_data.csv")
application_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


<h3>What variable(s) are considered the target(s) for your model?</h3>

Although STATUS and SPECIAL_CONSIDERATIONS are binary like IS_SUCCESSFUL, the purpose is to predict whether applicants will be successfull if funded.

<ul>
    <li>IS_SUCCESSFUL</li>
</ul>

<h3>What variable(s) are considered the feature(s) for your model?</h3>
<ul>
    <li>APPLICATION_TYPE</li>
    <li>AFFILIATION</li>
    <li>CLASSIFICATION</li>
    <li>USE_CASE</li>
    <li>ORGANIZATION</li>
    <li>INCOME_AMT</li>
    <li>ASK_AMT</li>
    <li>STATUS</li>
    <li>SPECIAL_CONSIDERATIONS</li>
</ul>

In [21]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
print(application_df.shape)
application_df.drop(['EIN','NAME'], axis=1, inplace=True)
print(application_df.shape)

(34299, 12)
(34299, 10)


In [22]:
# Determine the number of unique values in each column.
'''
for col in application_df:
    print(application_df[col].unique())
'''
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [23]:
# Look at APPLICATION_TYPE value counts for binning
application_df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [24]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Going with T10 @ 528 to be the cuttoff.  Next is T9 which is 156 and all others after are less so grouping them all...
application_types_to_replace = []

for app, cnt in application_df.APPLICATION_TYPE.value_counts().iteritems():
    if cnt < 528:
        application_types_to_replace.append(app)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [25]:
# Look at CLASSIFICATION value counts for binning
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [26]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()> 1]


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [27]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Going with c2100 @ 1883 to be the cuttoff.  Next is c7000 which is 777 and all others after are less so grouping them all...
classifications_to_replace = []

for cla, cnt in application_df.CLASSIFICATION.value_counts().iteritems():
    if cnt < 1883:
        classifications_to_replace.append(cla)

# Replace in dataframe
for cla in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cla,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [28]:
catColumns = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
catColumns

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [29]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_df = pd.get_dummies(application_df, columns=catColumns)
converted_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [30]:
# Split our preprocessed data into our features and target arrays
X = converted_df.drop('IS_SUCCESSFUL', axis=1).values
y = converted_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [31]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [32]:
from pandas._libs.tslibs import period

# Define callback for model weights
def createCheckPointCallback():
  fpath =  f"/content/drive/MyDrive/Homework/Deep-Learning/Model_Weights/"
  fname = "model_weights_{epoch}.hdf5"
  return ModelCheckpoint(filepath=fpath+fname,
                              verbose=1,
                              save_weights_only=True,
                              save_freq="epoch",
                              period=5)

## Compile, Train and Evaluate the Model

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = number_input_features  * 0.66
hidden_nodes_layer2 = number_input_features / 2
epochs=50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 28)                1232      
                                                                 
 dense_4 (Dense)             (None, 21)                609       
                                                                 
 dense_5 (Dense)             (None, 1)                 22        
                                                                 
Total params: 1,863
Trainable params: 1,863
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
checkPoint_callBack = createCheckPointCallback()
fit_model = nn.fit(X_train_scaled,y_train,epochs=epochs, callbacks=[checkPoint_callBack])

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5827 - accuracy: 0.7154
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5567 - accuracy: 0.7286
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5541 - accuracy: 0.7288
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.7305
Epoch 5/50
804/804 [==============================] - ETA: 0s - loss: 0.5505 - accuracy: 0.7308
Epoch 00005: saving model to /content/drive/MyDrive/Homework/Deep-Learning/Model_Weights/model_weights_5.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7308
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5496 - accuracy: 0.7306
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5488 - accuracy: 0.7308
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7318
Epoch

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5553 - accuracy: 0.7275 - 354ms/epoch - 1ms/step
Loss: 0.5552963018417358, Accuracy: 0.7274635434150696


In [37]:
# Export our model to HDF5 file
nn.save('/content/drive/MyDrive/Homework/Deep-Learning/Model_Weights/AlphabetSoupCharity.h5')